# Data Wrangling Dr. Uma 
12/10/2022

In [1]:
import pandas as pd

: 

: 

In [1]:
import pandas as pd
import numpy as np

: 

: 

In [1]:
# import data
data = pd.read_excel('testdata.xlsx')
data.head()

NameError: name 'pd' is not defined

In [5]:
# clean data
# fix column headers
data.columns = data.iloc[0, :]
# drop first row
data = data.drop(0, axis=0)
# forward fill missing data
data = data.ffill()
# select required columns
data = data.iloc[:, [0, 1, 2, 3]]
# fix columns names
data = data.rename(columns={'LENGTH\nOF\nCASE':'length of case'})
# show header
data.head()

,SUBJECT,LENGTH\nOF\nCASE,Monitor clock time,PSI,"PSI<25 i.e. burst suppression (1=Y, 2=N)",# of burst suppressions for case,Burst suppression duration each (sec),Burst suppression duration total for case (sec)
0,SUBJECT,LENGTH\nOF\nCASE,Monitor clock time,PSI,"PSI<25 i.e. burst suppression (1=Y, 2=N)",# of burst suppressions for case,Burst suppression duration each (sec),Burst suppression duration total for case (sec)
1,11,00:28:00,07:29:35,100,NaN,NaN,NaN,NaN
2,NaN,NaN,07:29:37,100,NaN,NaN,NaN,NaN
3,NaN,NaN,07:29:39,100,NaN,NaN,NaN,NaN
4,NaN,NaN,07:29:41,100,NaN,NaN,NaN,NaN


In [239]:
# copy data
data_ = data.copy()

# replace strings with NaNs
data_.PSI = data_.PSI.replace('-', np.nan)

# fill in missing values by forward filling
data_ = data_.ffill()

# add column to indicate regions where PSI drops below 25
data_['less than 25'] = (data_.PSI.astype(int) <= 25).astype(int)

# column to record number of burst supressions, burst duration
data_['number of burst supressions'] = np.nan
data_['burst_duration'] = np.nan

# convert clock time into sections
clock_list = np.array(data_['Monitor clock time'].astype(str).str.split(':').tolist()).astype(int).copy()
data_['seconds']  = clock_list[:, 0] * 3600 + clock_list[:, 1] * 60 + clock_list[:, 2]

# convert length of case into integer seconds
seconds_list = np.array(data_['length of case'].astype(str).str.split(':').tolist()).astype(int).copy()
data_['length of case'] = seconds_list[:, 1]

# sort values
data_.sort_values(['seconds', 'SUBJECT'])

# reset inded
data_.reset_index(inplace=True, drop=True)
data_.head()

,SUBJECT,length of case,Monitor clock time,PSI,less than 25,number of burst supressions,burst_duration,seconds
0,11,28,07:29:35,100.0,0,NaN,NaN,26975
1,11,28,07:29:37,100.0,0,NaN,NaN,26977
2,11,28,07:29:39,100.0,0,NaN,NaN,26979
3,11,28,07:29:41,100.0,0,NaN,NaN,26981
4,11,28,07:29:43,100.0,0,NaN,NaN,26983


Write functions to measure the number of bursts and burst duration

In [211]:
# count the number of contiguous regions
def contiguous_regions(x):
    count = 0
    j = 1
    for i in x['less than 25']:
        if j == 1:
            # i indicate the start of a region where PSI is less than 25
            if i == 1:
                count +=1
                j = 0
        # when continugous region ends switch j back to 1
        else:
            if i == 0:
                j = 1
    return count

def burst_duration(x):
    # get the duration of each burst
    time_delta = []
    j = 1
    for k, i in enumerate(x['less than 25']):
        if j == 1:
            # i indicate the start of a region where PSI is less than 25
            if i == 1:
                start_time = x.seconds[k]
                j = 0
        # when continugous region ends switch j back to 1
        else:
            if i == 0:
                time_delta.append(end_time - start_time)
                j = 1
        end_time = x.seconds[k]
    return time_delta


In [262]:
results = np.zeros((10000, 5), dtype='object')
j = 0
for i in data_.SUBJECT.unique():
    # get data correspondingto subject i
    x = data_[data_.SUBJECT == i].reset_index().copy()
    # get list of burst durations for subject i
    durations = burst_duration(x)
    # get the number of regions
    n_regions = contiguous_regions(x)
    # get total burst duration
    total_duration = sum(durations)
    # update array
    for duration in durations:
        results[j, 0] = str(i) # subject 
        results[j, 1] = int(j + 1)  # burst_id
        results[j, 2] = int(duration)  # burst duration
        results[j, 3] = int(n_regions)  # total bursts for subject 
        results[j, 4] = int(total_duration)  # total burst duration
        j += 1

# header of results
results[:10, :]

array([['11', 1, 124, 1, 124],
       ['12', 2, 36, 7, 350],
       ['12', 3, 38, 7, 350],
       ['12', 4, 54, 7, 350],
       ['12', 5, 48, 7, 350],
       ['12', 6, 90, 7, 350],
       ['12', 7, 40, 7, 350],
       ['12', 8, 44, 7, 350],
       ['13', 9, 36, 2, 36],
       ['14', 10, 2, 2, 10]], dtype=object)

In [3]:
# copy result into dataframe
results_ = pd.DataFrame(results).copy()

# drop extra rows allocated to array
results_ = results_[results_[0] != 0]

# label columns
results_.columns = ['subject', 'burst_id', 'burst suppression duration (s)', 'burst supression regions', 'total burst duration (s)']

# join length of case column to results
results_ = results_.merge(data_[['SUBJECT', 'length of case']].drop_duplicates(), left_on='subject', right_on='SUBJECT').drop('SUBJECT', axis=1)

results_.head()

NameError: name 'pd' is not defined

In [1]:
# save data
results_.to_csv('testdata-clean.csv')

NameError: name 'results_' is not defined

In [2]:
results_

NameError: name 'results_' is not defined